In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
INIT_LG = 1e-4
epochs = 20
bs = 32

directory = r"C:\Users\MOSTAFA MAHMOUD\Desktop\Mask Detection\dataset"
categories = ["with_mask","without_mask"]

data=[]
label = []

for cate in categories :
    path= os.path.join(directory,cate)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        label.append(cate)

C:\Users\MOSTAFA MAHMOUD\anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [6]:
lb = LabelBinarizer()
label = lb.fit_transform(label)
label = to_categorical(label)

data = np.array(data)
label = np.array(label)

(trainX, testX, trainY, testY) = train_test_split(data , label , test_size=0.20 , stratify=label , random_state=42)

In [7]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [8]:
beasModel = MobileNetV2(weights="imagenet",include_top=False,
                       input_tensor=Input(shape=(224,224,3)))
headmodel = beasModel.output
headmodel = AveragePooling2D(pool_size=(7,7))(headmodel)
headmodel = Flatten(name="flatten")(headmodel)
headmodel = Dense(128, activation="relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2, activation="softmax")(headmodel)

model = Model(inputs=beasModel.input, outputs=headmodel)

In [9]:
for layer in beasModel.layers:
    layer.trinable=False

opt = Adam(lr=INIT_LG , decay=INIT_LG/ epochs)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

H = model.fit(
    aug.flow(trainX, trainY, batch_size=bs),
    steps_per_epoch=len(trainX) // bs,
    validation_data=(testX,testY),
    validation_steps=len(testX) //bs,
    epochs=10
)

C:\Users\MOSTAFA MAHMOUD\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
95/95 [==============================] - 383s 4s/step - loss: 0.1909 - accuracy: 0.9394 - val_loss: 0.8527 - val_accuracy: 0.8070
Epoch 2/10
95/95 [==============================] - 371s 4s/step - loss: 0.0282 - accuracy: 0.9918 - val_loss: 0.5465 - val_accuracy: 0.8840
Epoch 3/10
95/95 [==============================] - 355s 4s/step - loss: 0.0170 - accuracy: 0.9941 - val_loss: 1.4221 - val_accuracy: 0.8214
Epoch 4/10
95/95 [==============================] - 355s 4s/step - loss: 0.0163 - accuracy: 0.9954 - val_loss: 0.7130 - val_accuracy: 0.8787
Epoch 5/10
95/95 [==============================] - 357s 4s/step - loss: 0.0108 - accuracy: 0.9967 - val_loss: 0.5767 - val_accuracy: 0.9061
Epoch 6/10
95/95 [==============================] - 367s 4s/step - loss: 0.0103 - accuracy: 0.9974 - val_loss: 0.7363 - val_accuracy: 0.8970
Epoch 7/10
95/95 [==============================] - 363s 4s/step - loss: 0.0045 - accuracy: 0.9980 - val_loss: 0.3396 - val_accuracy: 0.9400
Epoch 8/10
95

In [14]:
predIxs = model.predict(testX, batch_size=bs)

predIxs = np.argmax(predIxs, axis=1)

In [17]:
print(classification_report(testY.argmax(axis=1),predIxs,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       1.00      0.97      0.99       383
without_mask       0.97      1.00      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [18]:
model.save("mask_det.model",save_format="h5")

C:\Users\MOSTAFA MAHMOUD\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
